## Setup & Import

In [3]:
# 📦 Setup e import
import sys
from pathlib import Path

# Aggiunge src/ al PYTHONPATH (siamo in /notebooks/)
sys.path.append(str(Path("../src").resolve()))

from dfa_generator import (
    dfa_parity,
    dfa_contains,
    compose,
    simulate_samples,
)
from pta_builder import PTA

# ---------------------------------------------------------------------
# (ri-)Generiamo i campioni S⁺/S⁻ come nel notebook 01
dfa1 = dfa_parity("a", modulo=2)
dfa2 = dfa_contains("bb", alphabet={"a", "b"})
product = compose([dfa1, dfa2], accepting_policy="intersection")

S_pos, S_neg = simulate_samples(
    product,
    max_len=1000,
    n_positive=100,
    n_negative=100,
    seed=42
)

print(f"Campioni generati  ➜  |S⁺|={len(S_pos)}, |S⁻|={len(S_neg)}")


Campioni generati  ➜  |S⁺|=100, |S⁻|=100


## Costruzione PTA

In [ ]:
# Costruzione della PTA dai campioni
pta = PTA.build(S_pos, S_neg)  # costruisce l'albero
print(pta)                     # stampa: <PTA: |Q|=..., |Σ|=...>

# Visualizzazione: PNG se Graphviz disponibile, altrimenti DOT
Path("figs").mkdir(exist_ok=True)

try:
    pta.to_graphviz("figs/pta")
    print("✅  PNG PTA salvato in figs/pta.png")
except Exception as e:
    print("⚠️  PNG non generato, salvo DOT.")
    print(e)
    with open("figs/pta.dot", "w") as f:
        f.write(pta.to_dot())
    print("📝  DOT salvato in figs/pta.dot")

# ---------------------------------------------------------------------
# Verifica di coerenza con i campioni S⁺ / S⁻
assert all(pta.accepts(w) for w in S_pos), "Errore: qualche S⁺ non accettato"
assert all(not pta.accepts(w) for w in S_neg), "Errore: qualche S⁻ accettato"
print("✅  PTA coerente con i campioni S⁺ / S⁻")

<PTA: |Q|=101915, |Σ|=2>
